# Проект команды 306 - Проектный семинар - №3 (Извлечение признаков)

Состав команды:

1. Алиев Хайрутдин Аллилович
2. Зубов Дмитрий Сергеевич
3. Курбанов Иван Сергеевич
4. Лухнев Игорь Дмитриевич
5. Шишков Максим Алексеевич

## TL;DR
---
В данном документе мы проводим применение различных предобученных нейронных сетей для извлечения признаков из картинок.

---

## Предварительные зависимости
В следующей ячейке будут установлены необходимые библиотеки через `pip`

In [1]:
# Use !pip install -q [your-package]
!pip install -q PyGithub
!pip install -q fsspec

Следующая ячейка для импорта установленных пакетов в проект

In [2]:
# use aliases for long names
import os
import requests
from github import Github as gh
from getpass import getpass
import re
import pandas as pd
import numpy as np
import base64
from io import StringIO
from pathlib import Path
from PIL import Image
import torch
import torchvision
from tqdm.auto import tqdm
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torchvision import transforms
import os
import random
from IPython.display import clear_output
import matplotlib.pyplot as plt

Зафиксируем random_seed в нашем документе

In [3]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    
set_random_seed(306)

## Загрузка данных 

Запросим у пользователя креды для подключения к репозиторию

In [4]:
login = input('Enter your login: ')
password = getpass('Enter the secret value: ')

Подключимся к репозиторию

In [5]:
repo_path = "IgorLukhnev/FarFetchRS"
# Запишем токен в переменную окружения
g_token = os.getenv('GITHUB_TOKEN', password)
# Вополним коннект
g = gh(g_token)
# Подключимся к репо
repo = g.get_repo(repo_path)

Данные будем загружать на основе датасета, подготовленного на этапе EDA

In [6]:
dir_path = 'Data/simpleCat'
data_path = 'Data/simpleCat/final_data.csv'
contents = repo.get_contents(dir_path)
for x in contents:
    if x.path == data_path:
        data = pd.read_csv(StringIO(base64.b64decode(repo.get_git_blob(x.sha).content).decode("utf8"))).drop(columns=['Unnamed: 0'])

data.head()

## Датасет для моделей 

Для начала научимся работать с датасетом для переноса обучения - clothing-dataset-full (содержит 5000 изображений одежды по 20 лейблам). На данном датасете мы дообучим несколько моделей:
1. ResNet
2. Inception
3. VGG-16
4. MobileNet

Но это в перспективе - сейчас только ResNet

В таком случае создаем класс для датасета

In [7]:
class LearningDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform, data_labels):
        '''
        data_dir - путь к директории, где лежат изображения
        transform - объект для трансформации изображений
        data_labels - датафрейм для мэппинга картинок с классами
        '''
        self.data_dir = data_dir
        self.transform = transform
        self.data_labels = data_labels
        # Создаем словарь для перевода класса в цифровой формат
        l2int = {l: i for i, l in enumerate(data_labels['label'].unique().tolist())}
        self.l2int = l2int
    
    def __getitem__(self, idx):
        label = self.data_labels.iloc[idx, 2]
        img_name = self.data_labels.iloc[idx, 0]
        path = os.path.join(self.data_dir, img_name + '.jpg')
        with open(path, 'rb') as f:
            img = Image.open(f)
            img = img.convert('RGB')
            if self.transform is not None:
                img = self.transform(img)
            return img, self.l2int[label]
    
    def __len__(self):
        return self.data_labels.shape[0]

Так, подождите - нужно реализовать еще трансформер.

In [8]:
batch_size = 64

train_transform = transforms.Compose(
    [
     transforms.Resize(224), # меняем размер изображения
     transforms.RandomHorizontalFlip(p=0.5), # половину изображений перевернем по горизонтали
     transforms.RandomAdjustSharpness(sharpness_factor=2), # увеличим где-то резкость
     transforms.RandomAutocontrast(), # применим автоконтраст
     transforms.RandomCrop(200), # часть фотографий обрежем немного
     transforms.Resize(224), # часть фотографий и снова растянем
     transforms.ToTensor(), # преобразуем в тензор
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]) # нормализуем значения пикселей по цветовым слоям
    ]
    )


## Загрузка модели

Теперь мы готовы к загрузке модели и ее дообучению

Загружаем ResNet50 предобученную на ImageNet, заменяем выходной слой, чтобы он выдавал ответ на 20 значений, а не 1000

In [9]:
resnet = torchvision.models.resnet50(pretrained=True)

resnet.fc = nn.Linear(in_features=2048, out_features=20, bias=True)

print(resnet)

## Функции обучения

Напишем функцию для обучения 1 эпохи модели

In [10]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, device="cuda:0"):
    '''
    model - сама модель
    train_dataloader - загрузчик данных батчами
    criterion - функционал потерь
    optimizer - метод оптимизации функционала потерь
    device - процессор, на котором выполняются вычисления
    '''
    model.train()
    losses = 0
    for X_train, y_train in tqdm(train_dataloader):
        X_train = X_train.to(device)
        y_train = y_train.to(device)
        # обнуляем градиенты оптимизатора, чтобы он считал только по данному шагу
        optimizer.zero_grad()
        # получаем ответы модели
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        # считаем градиенты
        loss.backward()
        optimizer.step()
        losses += loss.data.item()
    return losses / len(train_dataloader)

Функция для предсказывания ответов модели

In [11]:
def predict(model, val_dataloader, criterion, device="cuda:0"):
    '''
    model - сама модель
    val_dataloader - загрузчик данных батчами
    criterion - функционал потерь
    device - процессор, на котором выполняются вычисления
    '''
    model.to(device)
    model.eval()
    losses = np.array([])
    predicted_classes = np.array([])
    true_classes = np.array([])
    with torch.no_grad():
        for X_val, y_val in tqdm(val_dataloader):
            X_val = X_val.to(device)
            y_val = y_val.to(device)
            y_pred = model(X_val)
            losses= np.append(losses, 
                              criterion(y_pred, y_val).to('cpu').detach().numpy())
            predicted_classes = np.append(predicted_classes, 
                                          torch.max(F.softmax(y_pred, dim=1), dim=1)[1].to('cpu').detach().numpy())
            true_classes = np.append(true_classes,
                                     y_val.to('cpu').detach().numpy())
    return losses, predicted_classes, true_classes

Ну и наконец, само обучение

In [12]:
def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=10, scheduler=None):
    '''
    model - сама модель
    train(val)_dataloader - загрузчик данных батчами
    criterion - функционал потерь
    optimizer - метод оптимизации функционала потерь
    device - процессор, на котором выполняются вычисления
    n_epochs - число эпох обучения
    scheduler - объект, уменьшающий шаг оптимизатора
    '''
    model.to(device)
    train_loss = []
    losses = []
    val_losses = []
    for epoch in range(n_epochs):
        train_loss = train_one_epoch(
            model, train_dataloader, criterion, optimizer, device=device)
        losses.append(train_loss)
        clear_output(True)
        losses_, preds, y_val = predict(model, val_dataloader, criterion, 
                                        device=device)

        val_loss = np.mean(losses_)
        val_losses.append(val_loss)
        val_accuracy = accuracy_score(y_val, preds)
        if scheduler is not None:
            scheduler.step(val_accuracy)
        plt.plot(range(epoch + 1), losses, marker='o', label='Train')
        plt.plot(range(epoch + 1), val_losses, c='r', marker='x', label='Val')
        plt.title(f'Cross-Entropy во время обучения')
        plt.xlabel('Эпоха')
        plt.ylabel('CE')
        plt.legend()
        plt.show()
        print(
            f'''{epoch + 1} эпоха; Train CE = {losses[-1]};\nVal CE = {val_losses[-1]};
Val accuracy = {val_accuracy}''')

### Дообучение

Инициализируем все объекты, которые нам пригодятся для обучения

In [39]:
bad_imgs = [
    'c60e486d-10ed-4f64-abab-5bb698c736dd', 
    '040d73b7-21b5-4cf2-84fc-e1a80231b202',
    '784d67d4-b95e-4abb-baf7-8024f18dc3c8',
    '1d0129a1-f29a-4a3f-b103-f651176183eb',
    'd028580f-9a98-4fb5-a6c9-5dc362ad3f09',
           ]

data_labels = pd.read_csv('../input/clothing-dataset-full/images.csv')

data_labels.drop(index=data_labels[data_labels['image'].apply(lambda x: x in bad_imgs)].index, inplace=True)

data_labels.head()

In [42]:
train_data_path = '../input/clothing-dataset-full/images_compressed'

optimizer = torch.optim.Adam(resnet.parameters())
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max')
n_epochs = 10
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

train_dataset = LearningDataset(train_data_path, transform=train_transform, data_labels=data_labels)

train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                               batch_size=batch_size, 
                                               shuffle=True)
val_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                             batch_size=batch_size,
                                             shuffle=False)

Запустим обучение на 10 эпохах

In [43]:
train(resnet, train_dataloader, val_dataloader, criterion, optimizer, device, n_epochs, scheduler)

Отлично, модель обучена - нужно сохранить ее конфигурацию, чтобы затем использовать в будущих документах

In [45]:
torch.save(resnet.state_dict(), 'resnet_on_clothing_tuned')

Окей, давайте пробовать получать значения признаков для наших данных

## Генерация признаков 

In [86]:
mask = (data['sex'] == 'm') & (data['cat'] == 'clothing') & (data['category1'] == 'trousers')

test_dir = '../input/farfetch-male-trousers/m_clothing_trousers'

test_labels = data.reset_index().loc[mask, 'index'].rename({'index': 'img'}, axis=1)
labels = test_labels.to_numpy().reshape(6412, 1)

In [57]:
# Берем все слои сети кроме последнего
modules = list(resnet.children())[:-1]
# Создаем генератор признаков
generator = nn.Sequential(*modules)
# Запрещаем накапливать градиенты
for p in generator.parameters():
    p.requires_grad = False

In [104]:
i = 0
for l in tqdm(labels):
    with open(f'../input/farfetch-male-trousers/m_clothing_trousers/{l.item()}.jpg', 'rb') as img_stream:
        img = Image.open(img_stream)
        img = img.convert('RGB')
        img = train_transform(img).unsqueeze(0).to(device)
        gens = generator(img)
        if i == 0:
            features = gens.squeeze(3).squeeze(2).squeeze(0).detach().to('cpu').numpy()
        else:
            features = np.vstack([features, gens.squeeze(3).squeeze(2).squeeze(0).detach().to('cpu').numpy()])
        i += 1
#         print(generator(img.unsqueeze(0).to(device)).squeeze(3).squeeze(2).squeeze(0).detach().to('cpu').numpy())

In [105]:
features.shape

In [123]:
new_features = pd.DataFrame(np.hstack([labels, features])).set_index(new_features[0].astype(int)).drop(columns=0)

In [126]:
to_save = new_features.drop(columns=new_features.sum()[new_features.sum() == 0].index)

In [127]:
to_save

In [115]:
initial_data = data.loc[mask, :]

In [132]:
final_data = pd.concat([initial_data, to_save], axis=1)

In [133]:
final_data.to_csv('data_image_embedding.csv', index=False)